# Baseline Test - 1D CNN
Google TPU v2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
folder_root = "/content/drive/MyDrive/ActigraphyTransformer/A-NEW/Baseline Tests"
folder_Data_2013 = "/content/drive/MyDrive/ActigraphyTransformer/A-NEW/Baseline Tests/Data_2013"
folder_1D_CNN = "/content/drive/MyDrive/ActigraphyTransformer/A-NEW/Baseline Tests/1D_CNN"

# Imports and Connects to TPU

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pyarrow fastparquet

In [ ]:
# @title Importing

# Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import os
from IPython.display import clear_output

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#from keras.layers.embeddings import Embedding
from keras.metrics import AUC

# Tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random

# Import Layers
from keras.layers import ConvLSTM2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling3D
from keras.layers import GlobalAveragePooling1D

In [ ]:
## SEEDS

# Hard Code Random Seeds.
r1 = 0
r2 = 1

# Set Random Seed
random.seed(r1)
tf.random.set_seed(r2)

In [ ]:
#@title Connect to TPU
print("TensorFlow version:", tf.__version__)

# Connect to the TPU cluster or fall back to CPU/GPU
try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Tries to connect to the TPU
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)
  devices = tf.config.list_logical_devices('TPU')
  print('TPU devices:', devices)
except ValueError:
  print("Could not connect to TPU; using CPU/GPU strategy instead.")
  strategy = tf.distribute.get_strategy()

# Example computation using the strategy
with strategy.scope():
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  @tf.function
  def matmul_fn(x, y):
    return tf.matmul(x, y)

  z = strategy.run(matmul_fn, args=(a, b))

print(z)

# Prepare the Data

## Define data mode (raw or smooth) and Task

In [ ]:
"""
Please Fill out Parameters Below
"""

Smoothing = False

Task = "SSRI"

Tasks = ["SSRI", "Benzodiazepine", "Sleep Strict", "Sleep Liberal", "Depression"] # pick a task from Tasks and set the "Task" variable in the above line


In [ ]:
if Smoothing:
  mode = "Smooth"
else:
  mode = "Raw"


if "Depression" in condition:
  train_sizes = [100, 250, 500, 1000, 2500, 2800]
  data_folder_location = os.path.join(folder_Data_2013, f"All_Meds_Depression/{mode}/TestSize2000_set1")

elif "Strict" in condition:
  train_sizes = [100, 250, 500, 1000, 2500, 3429]
  data_folder_location = os.path.join(folder_Data_2013, f"All_Meds_SleepDisorder_Strict/{mode}/TestSize2000_set1")

elif "Liberal" in condition:
  train_sizes = [100, 250, 500, 1000, 2500, 3429]
  data_folder_location = os.path.join(folder_Data_2013, f"All_Meds_SleepDisorder_Liberal/{mode}/TestSize2000_set1")

elif "Benzos" in condition:
  train_sizes = [100, 250, 500, 1000, 2500, 5769]
  data_folder_location = os.path.join(folder_Data_2013, f"All_Meds/{mode}/TestSize2000_set1")

elif "SSRI" in condition:
  train_sizes = [100, 250, 500, 1000, 2500, 5769]
  data_folder_location = os.path.join(folder_Data_2013, f"All_Meds_Taking_SSRI/{mode}/TestSize2000_set1")

else:
  raise ValueError("Invalid condition")

print(f"Current train sizes: {train_sizes}")
print(f"Data located at {data_folder_location}")

In [ ]:
test_size = 2000 # fixed

In [ ]:
# first save the test sets
X_test = np.load(os.path.join(data_folder_location, f'X_test_{test_size}.npy'))
y_test = np.load(os.path.join(data_folder_location, f'y_test_{test_size}.npy'))
print("successfully loaded X test and y test")
print(f"Shape of X test: {X_test.shape}")
print(f"Shape of y test: {y_test.shape}")

# standard scalar
train_scalar = StandardScaler()
train_scalar.fit(X_test)
X_test = train_scalar.transform(X_test)

Then, load the train and validation datasets by saving them into a dictionary

In [ ]:
train_sets = {}
val_sets = {}

In [ ]:
for size in train_sizes:
  X_train = np.load(os.path.join(data_folder_location, f'X_train_{size}.npy'))
  y_train = np.load(os.path.join(data_folder_location, f'y_train_{size}.npy'))
  train_sets[size] = (X_train, y_train)

  X_val = np.load(os.path.join(data_folder_location, f'X_val_{size}.npy'))
  y_val = np.load(os.path.join(data_folder_location, f'y_val_{size}.npy'))
  val_sets[size] = (X_val, y_val)

print("Data loaded successfully.")
print(f"Train set size: {len(train_sets)}")
print(f"Val set size: {len(val_sets)}")

### Sanity Checks

In [ ]:
for key, value in train_sets.items():
  print(f"For train size {key}: ")

  # print the shapes of X train and y train
  print(f"X train shape: {value[0].shape}")
  print(f"y train shape: {value[1].shape}")

  # also print the shapes of X val and y val
  print(f"X val shape: {val_sets[key][0].shape}")
  print(f"y val shape: {val_sets[key][1].shape}")

  print("================================")

# The 1D CNN Model

## Load the Model

In [ ]:
import tensorflow as tf

def create_model(X_train):
  model = Sequential()

  # add the first convolutional layer
  model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
  model.add(MaxPooling1D(pool_size=2)) # pooling
  # add the second convolutional layer
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  model.add(MaxPooling1D(pool_size=2)) # pooling
  model.add(GlobalAveragePooling1D()) # average pool
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5)) # prevent overfitting
  model.add(Dense(1, activation='sigmoid'))

  return model

### Since the 1D CNN Model depends on the size of X_train, we need to make a new model for every different X train size. Let's save them in a dictionary.

In [ ]:
models = {}

In [ ]:
# Compile the model -----

for train_size in train_sizes:
  X_train, _ = train_sets[train_size]

  print(f"====== training model on size {train_size} ======")
  print(f"X train shape: {X_train.shape}")

  with strategy.scope():
    train_model = create_model(X_train=X_train)
    train_model.compile(
      # Metrics
      loss= tf.keras.losses.BinaryCrossentropy(from_logits=False),
      metrics= tf.keras.metrics.AUC(name='auc'),
      # Optimizer
      optimizer= tf.keras.optimizers.Adam(
        learning_rate=0.00001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False
  ))

  models[train_size] = train_model

  print("MODEL SUMMARY: ", train_model.summary())
  print("\n")

In [ ]:
# inspect the model dictionary:
for key, value in models.items():
  print(f"Model for train size {key}:")
  print(value.summary())

## Prepare model weights and callbacks

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',    # monitor validation loss
    factor=0.5,            # reduce rate by a factor of 0.5
    patience=250,          # number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-4,           # minimum learning rate that the reduction can reach
    verbose=1
)

# earlyStopping callback
early_stopper = EarlyStopping(
    monitor='val_auc',  # monitor validation AUC
    mode='max',  # maximize AUC
    patience=250,  # number of epochs with no improvement after which training will be stopped
    verbose=1,  # display messages when early stopping is triggered
    restore_best_weights=True  # restore model weights from the epoch with the best value of the monitored quantity
)

## Train & Evaluate

In [ ]:
# model weights saving path
model_weight_path = os.path.join(folder_1D_CNN, f"Model Weights/All_Meds{condition}/{mode}")
print(f"current model weight path: {model_weight_path}")
if not os.path.exists(model_weight_path):
  os.makedirs(model_weight_path)

In [ ]:
epochs = 10000
batch_size = 64

In [ ]:
scores = {}
scores["test"] = {}
scores["val"] = {}

In [ ]:
model_histories = {}

In [ ]:
for size in train_sizes:

    print(f"\nSIZE:{size}")


    # Load X_train and fit
    X_train, y_train = train_sets[size]
    train_scalar = StandardScaler()
    train_scalar.fit(X_train)
    X_train = train_scalar.transform(X_train)

    # Load X_val and fit
    X_val, y_val = val_sets[size]
    val_scalar = StandardScaler()
    val_scalar.fit(X_val)
    X_val = val_scalar.transform(X_val)

    print("loaded X train and X val")

    # Set Class Weights = Balance
    class1 = sum(y_train)
    total = len(y_train)
    class0 = total-class1

    class_weights = {0: (class1/total),
                  1: ((class0/total))}

    print(f"class weights: {class_weights}")

    # get the corresponding model from the model dictionary
    train_model = models[size]
    print("model loaded")

    # Train model
    history = train_model.fit(
        X_train, y_train,
        epochs= epochs, # Edit
        batch_size= batch_size,
        validation_data = (X_val, y_val),
        shuffle=False,
        class_weight=class_weights,
        callbacks = [early_stopper, reduce_lr],
        verbose = 2)

    # save model history
    model_histories[size] = history

    # Save model
    current_model_name = f"1D_CNN_{size}.h5"
    current_model_weights_name = f"1D_CNN_{size}_weights.h5"
    print("current model name: ", current_model_name)
    train_model.save(os.path.join(model_weight_path, current_model_name))
    train_model.save_weights(os.path.join(model_weight_path, current_model_weights_name))
    print("model and weights saved")

    # Test model
    test_scores = train_model.evaluate(X_test, y_test, batch_size=64) # Test Set
    scores["test"][size] = test_scores[1]
    print("Test AUC:", test_scores[1])

    val_scores = train_model.evaluate(X_val, y_val, batch_size=64) # Val Set

    scores["val"][size] = val_scores[1]
    print("Val AUC:", val_scores[1])

In [ ]:
# let's look at the scores
for key, value in scores.items():
  print(f"{key}: {value}")

In [ ]:
# save all results in a .txt
print("\n\n")
print(scores)

results_path = os.path.join(folder_1D_CNN, f"results{condition}_{mode}.txt")

try:
    file_to_write = open(results_path, 'wt')
    file_to_write.write(str(scores))
    file_to_write.close()
    print("Successfully wrote to file")

except:
    print("Unable to write to file")

# Inspect the Model

In [ ]:
model_histories_plot_path = os.path.join(folder_1D_CNN, f"History Plots/All_Meds{condition}/{mode}")
# make the directory if it doesn't exist
if not os.path.exists(model_histories_plot_path):
  os.makedirs(model_histories_plot_path)

In [ ]:
#inspect model histories
for key, value in model_histories.items():
  print(f"Model history for train size {key}:")
  print(value)

In [ ]:
# Save each plot individually
for size, history in model_histories.items():
    print(f"Currently plotting graph with size {size}")

    plt.style.use('ggplot')
    plt.figure(figsize=(14, 5))
    plt.title(f"AUC over Epochs, Train Size {size}")
    plt.ylabel("AUC")
    plt.xlabel("Epochs")
    plt.plot(history.history['val_auc'], label='Validation AUC')
    plt.plot(history.history['auc'], label='Train AUC')
    plt.legend()

    # Save the plot
    plot_path = os.path.join(model_histories_plot_path, f"AUC_Over_Epochs_TrainSize{size}.png")
    plt.savefig(plot_path)  # Saves as PNG
    print(f"Saved figure at {plot_path}")
    plt.close()  # Close the plot to free up memory

In [ ]:
# save as one integrated plot

num_models = len(model_histories)
cols = 3  # number of columns in subplot grid
rows = (num_models + cols - 1) // cols  # calculate required number of rows

fig, axes = plt.subplots(rows, cols, figsize=(14 * cols, 5 * rows))
fig.suptitle("AUC over Epochs by Train Size", fontsize = 20)

for idx, (size, history) in enumerate(model_histories.items()):
    ax = axes[idx // cols, idx % cols]
    ax.set_title(f"Train Size {size}")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("AUC")
    ax.plot(history.history['val_auc'], label='Validation AUC')
    ax.plot(history.history['auc'], label='Train AUC')
    ax.legend()

# adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.95])

# save the combined plot
plot_path = os.path.join(model_histories_plot_path, "Combined_AUC_Over_Epochs.png")
plt.savefig(plot_path)
print(f"Saved combined figure at {plot_path}")
plt.close()  # close the plot